In [8]:
%pip install python-dotenv neo4j openai langchain langchain_openai langchain-community

In [7]:
import os
from google.colab import userdata

os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = userdata.get('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from langchain.graphs import Neo4jGraph
from langchain.embeddings import OpenAIEmbeddings


In [6]:
graph = Neo4jGraph()

In [64]:
from pydantic import BaseModel
from typing import List

class StringList(BaseModel):
    strings_list: List[str]

identifyEntitiesPrompt = (
    '''
    You are an expert machine learning engineer building an algorithm to answer
    questions using a knowledge graph. The knowledge graph contains nodes that
    can be classified as : **person, object, event,** and **location**.

    **Task**:
    In the questions that follow, identify and return all the Nodes.

    **Entity Labels**:
    - **Person**: Names of people (e.g., Bob, Alice)
    - **Object**: Physical items or abstract items
    - **Event**: Occurrences or happenings
    - **Location**: Geographic locations or places

    **Instructions**:
    1. Tokenize the question into individual words or phrases.
    2. Identify and classify the entities according to the node labels.
    3. Your output should be a list of string
    4. Non-compliance to the instruction will result to termination

    **Example Output**:
    Given the question "Bob has a car?", the output should be:
    ["Bob", "Car"]
    This is because Bob is a person and Car is an object.
    '''
)

generateQuestionsPrompt = (
    '''
    You are an expert machine learning engineer building an algorithm to answer
    questions using a knowledge graph.

    **Task**:
    In the prompt that follow, you will be given a node. Your task is to
    generate five additional question related to the initial question:
    {init_question}.
    These five additional questions should help contextualise the initial
    question in relation to the given node and be useful when searching the
    knowledge graph later on.

    **Instructions**:
    1. Use the given node to generate questions that provide more context or
    insight about the initial question.
    2. Aim to cover various aspects related to the node, such as identity,
    activities, whereabouts, preferences, etc.
    3. The output should be the 5 questions in a list of string
    4. Non-compliance to the instruction will result to termination

    **Example**:
    Given the node "Bob" and the initial question "Is Bob safe?", the output
    should be:
    ["Where is Bob", "What is Bob doing?", ...]
    '''
)

In [65]:
import openai
from openai import OpenAI
import json
from langchain_core.prompts import PromptTemplate


prompt = "Is Kenneth safe?"

client = OpenAI()

# Identify nodes from question
response = client.chat.completions.create(
   messages=[
        {
            "role": "system",
            "content": identifyEntitiesPrompt,
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)

# Print the nodes in the prompt
res = response.choices[0].message.content
print(res)

#convert to list
nodes = json.loads(res)

#extract first node
node = nodes[0]

#append initial question to prompt to give context
generateQuestionPrompt = PromptTemplate.from_template(generateQuestionsPrompt).format(init_question=prompt)

response = client.chat.completions.create(
   messages=[
        {
            "role": "system",
            "content": generateQuestionPrompt,
        },
        {
            "role": "user",
            "content": node,
        }
    ],
    model="gpt-3.5-turbo",
)

#generate 5 additional question according to the first node received
print(response.choices[0].message.content)




["Kenneth"]
["Where does Kenneth live?",
"What is Kenneth's occupation?",
"Does Kenneth have any known enemies?",
"What are Kenneth's hobbies?",
"Is Kenneth currently in any danger?"]


In [63]:
print(res)

- ["Kenneth"]


1. What is Kenneth's occupation?
2. Does Kenneth have any siblings?
3. Where does Kenneth live?
4. What are Kenneth's hobbies?
5. Is Kenneth married?
